In [1]:
# condig:utf-8
import torch as th
import numpy as np
class GM():
 
    def __init__(self):
        # 判断是否可用 gpu 编程 , 大量级计算使用GPU
        self._is_gpu = False  # th.cuda.is_available()
 
    def fit(self,dt:list or np.ndarray):
        self._df :th.Tensor = th.from_numpy(np.array(dt,dtype=np.float32))
 
        if self._is_gpu:
            self._df.cuda()
 
        self._n:int = len(self._df)
 
        self._x,self._max_value = self._sigmod(self._df)
 
        z:th.Tensor = self._next_to_mean(th.cumsum(self._x,dim=0))
 
        self.coef:th.Tensor = self._coefficient(self._x, z)
 
        del z
 
        self._x0:th.Tensor = self._x[0]
 
        self._pre:th.Tensor = self._pred()
 
    # 归一化
    def _sigmod(self,x:th.Tensor):
        _maxv:th.Tensor = th.max(x)
        return th.div(x,_maxv),_maxv
 
    # 计算紧邻均值数列
    def _next_to_mean(self, x_1:th.Tensor):
 
        z:th.Tensor = th.zeros(self._n-1)
        if self._is_gpu:
            z.cuda()
 
        for i in range(1,self._n):  # 下标从0开始，取不到最大值
            z[i - 1] = 0.5 * x_1[i] + 0.5 * x_1[i - 1]
 
        return z
 
    # 计算系数 a,b
    def _coefficient(self,x:th.Tensor,z:th.Tensor):
 
        B:th.Tensor = th.stack((-1*z, th.ones(self._n-1)),dim=1)
 
        Y:th.Tensor = th.tensor(x[1:],dtype=th.float32).reshape((-1,1))
 
        if self._is_gpu:
            B.cuda()
            Y.cuda()
 
        # 返回的是a和b的向量转置，第一个是a 第二个是b；
        return th.matmul(th.matmul(th.inverse(th.matmul(B.t(), B)), B.t()),Y)
 
 
    def _pred(self,start:int=1,end:int=0):
 
        les:int = self._n+end
 
        resut:th.Tensor = th.zeros(les)
 
        if self._is_gpu:
            resut.cuda()
 
        resut[0] = self._x0
 
        for i in range(start,les):
            resut[i] = (self._x0 - (self.coef[1] / self.coef[0])) * \
                            (1 - th.exp(self.coef[0])) * th.exp(-1 * self.coef[0] * (i))
        del les
        return resut
 
    # 计算绝对误差
    def confidence(self):
        return round((th.sum(th.abs(th.div((self._x-self._pre),self._x)))/self._n).item(),4)
 
    # 预测个数，默认个数大于等于0，
    def predict(self,m:int=1,decimals:int=4):
 
        y_pred:th.Tensor = th.mul(self._pre,self._max_value)
 
        y_pred_ = th.zeros(1)
 
        if m<0:
            return "预测个数需大于等于0"
        elif m>0:
            y_pred_:th.Tensor = self._pred(self._n,m)[-m:].mul(self._max_value)
        else:
            if self._is_gpu:
                return list(map(lambda _: round(_, decimals), y_pred.cpu().numpy().tolist()))
            else:
                return list(map(lambda _:round(_,decimals),y_pred.numpy().tolist()))
 
        # cat 拼接 0 x水平拼接，1y垂直拼接
        result:th.Tensor = th.cat((y_pred,y_pred_),dim=0)
 
        del y_pred,y_pred_
 
        if self._is_gpu:
            return list(map(lambda _: round(_, decimals), result.cpu().numpy().tolist()))
 
        return list(map(lambda _:round(_,decimals),result.numpy().tolist()))
 
 
 
if __name__=="__main__":
    ls = np.arange(91,100,2)
    print(type(ls))
    # ls = list(range(91, 100, 2))
    gm = GM()
    gm.fit(ls)
    print(gm.confidence())
    print(ls)
    print(gm.predict(m=2))

<class 'numpy.ndarray'>


<ipython-input-1-60b02699fad1>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y:th.Tensor = th.tensor(x[1:],dtype=th.float32).reshape((-1,1))


0.0002
[91 93 95 97 99]
[91.0, 93.0178, 94.9758, 96.9751, 99.0164, 101.1007, 103.2289]


In [2]:
import pandas as pd 
from IPython.display import display
data = pd.read_excel('regression.xlsx')
data = data[['Country','Year','Score']]
data

,Country,Year,Score
0,Afghanistan,2000,4.057341
1,Albania,2000,3.658972
2,Algeria,2000,3.405881
3,Angola,2000,2.791749
4,Antigua and Barbuda,2000,2.937933
...,...,...,...
3755,Venezuela (Bolivarian Republic of),2019,3.302967
3756,Viet Nam,2019,3.093148
3757,Yemen,2019,3.290519
3758,Zambia,2019,3.611598


In [3]:
confidence =[]
all_result = pd.DataFrame()
for country in data.Country.unique():
    print(country)
    one_country_data = data[data.Country == country]
    one_country_result = []
    country_li = []
    year_li = []
    for year in range(2000,2026):
        country_li.append(country)
        year_li.append(year)
    ls = np.array(one_country_data['Score'])
    gm = GM()
    gm.fit(ls)
    print(gm.confidence())
    confidence.append(gm.confidence())
    pred = gm.predict(m=6)
    one_country_result.append(country_li)
    one_country_result.append(year_li)
    one_country_result.append(pred)
    one_country_result = pd.DataFrame(one_country_result)
    one_country_result =one_country_result.T
    all_result = pd.concat([all_result,one_country_result],axis=0)
all_result

Afghanistan
0.0626
Albania
0.0368
Algeria
0.0274
Angola
0.064
Antigua and Barbuda
0.0226
Argentina
0.0364
Armenia
0.0167
Australia
0.0454
Austria
0.0054
Azerbaijan
0.0333
Bahamas
0.0216
Bahrain
0.0093
Bangladesh
0.0359
Barbados
0.0003
Belarus
0.016
Belgium
0.0182
Belize
0.0202
Benin
0.0684
Bhutan

<ipython-input-1-60b02699fad1>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y:th.Tensor = th.tensor(x[1:],dtype=th.float32).reshape((-1,1))



0.0602
Bolivia (Plurinational State of)
0.0073
Bosnia and Herzegovina
0.0108
Botswana
0.0075
Brazil
0.0457
Brunei Darussalam
0.0086
Bulgaria
0.0069
Burkina Faso
0.063
Burundi
0.0969
Cabo Verde
0.0103
Cambodia
0.0676
Cameroon
0.0576
Canada
0.0969
Central African Republic
0.0833
Chad
0.0645
Chile
0.0864
China
0.95
Colombia
0.1227
Comoros
0.0418
Congo
0.0087
Cook Islands
0.0232
Costa Rica
0.0175
Croatia
0.0572
Cuba
0.0711
Cyprus
0.0036
Czechia
0.0081
Democratic People's Republic of Korea
0.0085
Democratic Republic of the Congo
0.0223
Denmark
0.0166
Djibouti
0.0241
Dominica
0.0307
Dominican Republic
0.0498
Ecuador
0.0446
Egypt
0.2468
El Salvador
0.0807
Equatorial Guinea
0.0216
Eritrea
0.009
Estonia
0.0066
Eswatini
0.0049
Ethiopia
0.021
Faroe Islands
0.0088
Fiji
0.0778
Finland
0.0048
France
0.0194
French Polynesia
0.0247
Gabon
0.0106
Gambia
0.0214
Georgia
0.0196
Germany
0.0879
Ghana
0.0506
Greece
0.0456
Grenada
0.0087
Guatemala
0.057
Guinea
0.0487
Guinea-Bissau
0.0239
Guyana
0.0031
Haiti
0

,0,1,2
0,Afghanistan,2000,4.0573
1,Afghanistan,2001,4.3041
2,Afghanistan,2002,4.1269
3,Afghanistan,2003,3.9571
4,Afghanistan,2004,3.7943
...,...,...,...
21,Zimbabwe,2021,3.7301
22,Zimbabwe,2022,3.7295
23,Zimbabwe,2023,3.729
24,Zimbabwe,2024,3.7285


In [4]:
confidence=pd.DataFrame(confidence)
confidence.to_csv('confidence.csv')

In [5]:
all_result.columns = ['Country','Year','Score']
all_result

,Country,Year,Score
0,Afghanistan,2000,4.0573
1,Afghanistan,2001,4.3041
2,Afghanistan,2002,4.1269
3,Afghanistan,2003,3.9571
4,Afghanistan,2004,3.7943
...,...,...,...
21,Zimbabwe,2021,3.7301
22,Zimbabwe,2022,3.7295
23,Zimbabwe,2023,3.729
24,Zimbabwe,2024,3.7285


In [6]:
result = []
for country in data.Country.unique():
    one_country_data = all_result[
        (all_result['Country'] == country) &
        (all_result['Year'].isin([2020,2025]))
                                 ]
    target = 5 - one_country_data.iloc[0,2]
    change = one_country_data.iloc[1,2] - one_country_data.iloc[0,2]
    print(target ,  ' ',change)
    if change == 0:
        year = 'no change'
    else:
        year = target / change * 5
    result.append([country,year])
result = pd.DataFrame(result,columns=['Country','Year'])
result

3.0629999999999997   -0.3671
1.8723   -0.12769999999999992
1.0375   0.17979999999999974
1.1985999999999999   0.43429999999999946
2.558   -0.08810000000000029
2.3796   -0.11250000000000027
1.6568999999999998   -0.027300000000000324
2.8618   -0.18079999999999985
2.014   0.011399999999999633
2.9151   -0.11900000000000022
2.6506   -0.08820000000000006
1.9192   -0.08539999999999992
3.2724   -0.03750000000000009
2.0026   -0.0012999999999996348
1.5225   0.011400000000000077
1.9363000000000001   0.14720000000000022
1.7153   -0.10639999999999983
1.2334   0.3866999999999998
2.3281   -0.21309999999999985
2.2674   -0.17390000000000017
2.5777   -0.05580000000000007
1.6112000000000002   -0.05169999999999986
2.832   0.053199999999999914
1.5966   -0.054300000000000015
0.8194999999999997   0.08399999999999963
1.3466999999999998   0.0129999999999999
3.6564   -0.2598999999999998
1.5966999999999998   -0.04330000000000034
1.8407   0.15370000000000017
2.0048   -0.27070000000000016
2.7521   -0.1995
3.6959999

,Country,Year
0,Afghanistan,-41.7189
1,Albania,-73.3085
2,Algeria,28.8515
3,Angola,13.7992
4,Antigua and Barbuda,-145.176
...,...,...
183,Venezuela (Bolivarian Republic of),-93.8319
184,Viet Nam,136.916
185,Yemen,35.4629
186,Zambia,2687.04


In [7]:
result.to_csv('Year.csv')